In [67]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import os

In [57]:
ISK = 2000000000  # 2 billion ISK
MAX_EXPOSURE = 0.15 # 15% of isk allocated per item type

In [58]:
fpath = "data/forge_market_history.pkl"
df: pd.DataFrame = pd.read_pickle(fpath)
df["date"] = pd.to_datetime(df["date"])
df

,average,date,highest,lowest,order_count,volume,type_id,type_name
0,63.33,2016-01-01,66.00,63.31,89,10775285,18,Plagioclase
1,4500.02,2016-01-01,4500.02,4500.02,5,376,19,Spodumain
2,249.00,2016-01-01,249.99,230.58,111,1239579,20,Kernite
3,499.01,2016-01-01,499.01,499.01,11,5632,21,Hedbergite
4,3157.13,2016-01-01,3157.13,3157.13,15,19560,22,Arkonor
...,...,...,...,...,...,...,...,...
30449808,1689333.33,2026-01-15,1690000.00,1688000.00,3,3,91412,"For Those Lost, But Never Forgotten"
30449809,255820.00,2026-01-15,257200.00,250500.00,4,5,91413,Canned Quantum Superposition Wormhole
30449810,215275.00,2026-01-15,215700.00,214000.00,3,4,91414,Scrap of paper
30449811,101571.43,2026-01-15,102000.00,101000.00,6,7,91415,The Capsuleer’s Confectionary Codex – Field Ed...


In [59]:
# for the max date, get the market cap of each item. Sort high to low.
max_date = df["date"].max()
df["market_cap"] = df["average"] * df["volume"]
df_max_date: pd.DataFrame = df[df["date"] == max_date][["type_id", "type_name", "average", "volume", "market_cap"]].copy()
df_max_date = df_max_date.sort_values(by="market_cap", ascending=False)
# calculate market cap as a percent of total
total_market_cap = df_max_date["market_cap"].sum()
df_max_date["market_cap_percent"] = df_max_date["market_cap"] / total_market_cap
# do a running sum of market cap percent
df_max_date["market_cap_percent_cumsum"] = df_max_date["market_cap_percent"].cumsum()
df_max_date["pct_of_isk"] = df_max_date["average"] / ISK
df_max_date = df_max_date[(
    (df_max_date["market_cap_percent_cumsum"] <= 0.75)
    & (df_max_date["pct_of_isk"] <= MAX_EXPOSURE)
)]
max_date_type_ids = set(df_max_date["type_id"].unique())
df_max_date

,type_id,type_name,average,volume,market_cap,market_cap_percent,market_cap_percent_cumsum,pct_of_isk
30444242,25619,Logic Circuit,2.145000e+06,60174,1.290732e+11,0.008705,0.196099,0.001073
30443259,19421,23rd Tier Overseer's Personal Effects,1.338370e+08,940,1.258068e+11,0.008484,0.204583,0.066919
30447143,48121,Triglavian Survey Database,9.560000e+04,1104656,1.056051e+11,0.007122,0.211705,0.000048
30444964,30747,Sleeper Drone AI Nexus,5.000000e+06,20669,1.033450e+11,0.006969,0.225665,0.002500
30441682,11399,Morphite,1.875000e+04,5252539,9.848511e+10,0.006642,0.232306,0.000009
...,...,...,...,...,...,...,...,...
30443808,22291,Ballistic Control System II,9.864000e+05,5279,5.207206e+09,0.000351,0.748476,0.000493
30442812,17482,Strip Miner I,3.976000e+06,1309,5.204584e+09,0.000351,0.748827,0.001988
30440770,3057,Mega Pulse Laser II,5.561000e+06,935,5.199535e+09,0.000351,0.749177,0.002780
30441836,12019,Sacrilege,1.849107e+08,28,5.177500e+09,0.000349,0.749527,0.092455


In [60]:
df_grouped = df[df["date"] >= max_date - pd.Timedelta(days=365)].groupby("type_id").agg(
    {
        "date": ["count", "min", "max"], 
        "volume": "mean",
        "average": "mean",
        })
df_grouped

date                              volume       average
        count        min        max          mean          mean
type_id                                                        
18        366 2025-01-15 2026-01-15  2.504497e+06  4.202011e+01
19        302 2025-01-15 2026-01-15  1.007401e+03  1.275945e+04
20        366 2025-01-15 2026-01-15  2.645701e+06  8.449708e+02
21        349 2025-01-15 2026-01-15  6.993338e+03  1.108421e+03
22        366 2025-01-15 2026-01-15  7.278489e+03  5.062407e+03
...       ...        ...        ...           ...           ...
91413      37 2025-12-10 2026-01-15  3.521622e+01  6.930207e+05
91414      37 2025-12-10 2026-01-15  2.948649e+01  6.999442e+05
91415      38 2025-12-09 2026-01-15  2.800000e+01  3.006940e+05
91416      38 2025-12-09 2026-01-15  2.731579e+01  4.659849e+05
91516      30 2025-12-12 2026-01-13  5.200000e+00  6.805133e+09

[17270 rows x 5 columns]

In [61]:
df_filtered = df_grouped[(
    (df_grouped["date"]["count"] >= 300)
    & (df_grouped["volume"]["mean"] >= 100)
    & (df_grouped["date"]["max"] == max_date)
)]
df_filtered = df_filtered.reset_index()
filtered_type_ids = set(df_filtered["type_id"].unique())
df_filtered

type_id  date                              volume       average
             count        min        max          mean          mean
0         18   366 2025-01-15 2026-01-15  2.504497e+06  4.202011e+01
1         19   302 2025-01-15 2026-01-15  1.007401e+03  1.275945e+04
2         20   366 2025-01-15 2026-01-15  2.645701e+06  8.449708e+02
3         21   349 2025-01-15 2026-01-15  6.993338e+03  1.108421e+03
4         22   366 2025-01-15 2026-01-15  7.278489e+03  5.062407e+03
...      ...   ...        ...        ...           ...           ...
2709   85746   366 2025-01-15 2026-01-15  2.870713e+06  9.110164e+03
2710   85747   365 2025-01-15 2026-01-15  1.600110e+02  1.656601e+07
2711   87318   351 2025-01-30 2026-01-15  4.266410e+02  8.938031e+04
2712   87319   351 2025-01-30 2026-01-15  4.226097e+02  4.763037e+04
2713   87320   351 2025-01-30 2026-01-15  3.070028e+02  1.957737e+05

[2714 rows x 6 columns]

In [62]:
df_final = df[(
    (df["type_id"].isin(filtered_type_ids))
    & (df["type_id"].isin(max_date_type_ids))
)][[
    "type_id",
    "type_name",
    "date",
    "average",
    "volume",
]].copy()
df_final = df_final.sort_values(by=["type_id", "date"])
df_final

,type_id,type_name,date,average,volume
5,34,Tritanium,2016-01-01,6.31,12849216082
7568,34,Tritanium,2016-01-02,6.31,13418867991
15137,34,Tritanium,2016-01-03,6.30,9007209023
22764,34,Tritanium,2016-01-04,6.27,10186170799
30103,34,Tritanium,2016-01-05,6.24,12529870933
...,...,...,...,...,...
30408726,85746,Infomorph Decryption Key,2026-01-11,6061.00,1873428
30419249,85746,Infomorph Decryption Key,2026-01-12,6079.00,2054919
30429411,85746,Infomorph Decryption Key,2026-01-13,6105.00,2350821
30439430,85746,Infomorph Decryption Key,2026-01-14,6056.00,974031


In [68]:
# Make images directory
os.makedirs("img", exist_ok=True)
# Make plots of forecasts for each type_id.
# Save the images to the img/ directory.
for (type_id, type_name), x in df_final.groupby(["type_id", "type_name"]):
    df_prophet = x[["date", "average"]].rename(columns={"date": "ds", "average": "y"})
    model = Prophet()
    model.fit(df_prophet)
    # model.plot_components(model.predict(df_prophet))
    # plt.savefig(f"img/{type_id}_components.png")
    # plt.close()
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    fig = model.plot(forecast)
    fig.suptitle(f"{type_name} ({type_id})", fontsize=16)
    plt.savefig(f"img/{type_id}_forecast.png")
    plt.close()
    break

21:53:42 - cmdstanpy - INFO - Chain [1] start processing
21:53:42 - cmdstanpy - INFO - Chain [1] done processing


In [101]:
# Now do a simulation. Forecast the future prices. Buy low, sell high. Add up the total profit.

TEST_START_DATE = pd.to_datetime("2024-01-01")
HIGH_LOW_MULT = 1.0
for (type_id, type_name), data in df_final.groupby(["type_id", "type_name"]):
    # train the prophet model on the data before TEST_START_DATE
    train_data = data[data["date"] < TEST_START_DATE]
    test_data = data[data["date"] >= TEST_START_DATE]
    df_prophet = train_data[["date", "average"]].rename(columns={"date": "ds", "average": "y"})
    model = Prophet()
    model.fit(df_prophet)
    # make future dataframe for the test period
    future = model.make_future_dataframe(periods=7, include_history=False)
    forecast = model.predict(future)
    # create a buy order at the yhat_lower and see how long it takes to get filled.
    min_yhat = forecast["yhat"].min()
    min_yhat_lower = forecast["yhat_lower"].min()
    buy_price = min_yhat - (min_yhat - min_yhat_lower) * HIGH_LOW_MULT
    buy_complete_date = test_data[test_data["average"] <= buy_price]["date"].min()
    if pd.isna(buy_complete_date):
        print(f"Could not buy {type_name}")
        continue
    print(f"Bought {type_name} ({type_id}) at {buy_price:.2f} on {buy_complete_date.date()}")
    # train the model again up to the buy_complete_date
    train_data = data[data["date"] < buy_complete_date]
    test_data = data[data["date"] >= buy_complete_date]
    df_prophet = train_data[["date", "average"]].rename(columns={"date": "ds", "average": "y"})
    model = Prophet()
    model.fit(df_prophet)
    # make future dataframe for the test period
    future = model.make_future_dataframe(periods=7, include_history=False)
    forecast = model.predict(future)
    # create a sell order at the yhat_upper and see how long it takes to get
    max_yhat = forecast["yhat"].max()
    max_yhat_upper = forecast["yhat_upper"].max()
    sell_price = max_yhat + (max_yhat_upper - max_yhat) * HIGH_LOW_MULT
    sell_complete_date = test_data[test_data["average"] >= sell_price]["date"].min()
    if pd.isna(sell_complete_date):
        print(f"Could not sell {type_name}")
        continue
    print(f"Sold {type_name} ({type_id}) at {sell_price:.2f} on {sell_complete_date.date()}")
    profit_percent = (sell_price - buy_price) / buy_price * 100
    print(f"Profit: {profit_percent:.2f}%")
    # break

22:28:49 - cmdstanpy - INFO - Chain [1] start processing
22:28:49 - cmdstanpy - INFO - Chain [1] done processing
22:28:49 - cmdstanpy - INFO - Chain [1] start processing


Could not buy Tritanium


22:28:50 - cmdstanpy - INFO - Chain [1] done processing
22:28:50 - cmdstanpy - INFO - Chain [1] start processing


Bought Pyerite (35) at 9.69 on 2024-01-11


22:28:50 - cmdstanpy - INFO - Chain [1] done processing
22:28:50 - cmdstanpy - INFO - Chain [1] start processing


...

22:29:37 - cmdstanpy - INFO - Chain [1] done processing
22:29:37 - cmdstanpy - INFO - Chain [1] start processing


Bought Cybernetic Subprocessor - Standard (10221) at 21028112.98 on 2024-01-10


22:29:37 - cmdstanpy - INFO - Chain [1] done processing
22:29:38 - cmdstanpy - INFO - Chain [1] start processing


Sold Cybernetic Subprocessor - Standard (10221) at 23796638.36 on 2024-12-13
Profit: 13.17%


KeyboardInterrupt: 

In [95]:
test_data[test_data["average"] >= sell_price]["date"].min()

Timestamp('2024-03-21 00:00:00')

In [92]:
test_data[test_data["average"] <= buy_price]["date"].min()

Timestamp('2025-05-04 00:00:00')

In [90]:
buy_price

np.float64(3.9857994183997585)